In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the dataset
file_path = 'tsla.csv'
data = pd.read_csv(file_path)

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Set 'Date' as the index
data.set_index('Date', inplace=True)

# Display the first few rows of the dataset and the data types of each column
print(data.head())
print(data.dtypes)


                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2010-06-29  1.266667  1.666667  1.169333  1.592667   1.592667  281494500
2010-06-30  1.719333  2.028000  1.553333  1.588667   1.588667  257806500
2010-07-01  1.666667  1.728000  1.351333  1.464000   1.464000  123282000
2010-07-02  1.533333  1.540000  1.247333  1.280000   1.280000   77097000
2010-07-06  1.333333  1.333333  1.055333  1.074000   1.074000  103003500
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


In [2]:
# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))


In [3]:
# Prepare the training and validation data
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - 60:]  # Include the last 60 days of training data for context


In [4]:
# Create sequences for training
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

seq_length = 60
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)


In [5]:
# Reshape the data to be suitable for LSTM input
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [6]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))  # Predicting 1 value

model.compile(optimizer='adam', loss='mean_squared_error')


C:\Users\zains\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 23s 102ms/step - loss: 3.6466e-04 - val_loss: 0.0055
Epoch 2/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - loss: 2.9458e-05 - val_loss: 0.0027
Epoch 3/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - loss: 1.8367e-05 - val_loss: 0.0024
Epoch 4/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - loss: 1.3573e-05 - val_loss: 0.0021
Epoch 5/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - loss: 1.5471e-05 - val_loss: 0.0050
Epoch 6/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - loss: 1.2378e-05 - val_loss: 0.0020
Epoch 7/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - loss: 1.4408e-05 - val_loss: 0.0029
Epoch 8/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - loss: 1.2444e-05 - val_loss: 0.0024
Epoch 9/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - loss: 1.0708e-05 - val_loss: 0.0018
Epoch 10/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 1.1430e-05 - val_loss: 0.0017


In [8]:
# Make predictions on the test data
predicted_scaled = model.predict(X_test)

# Inverse transform to get the actual values
predicted = scaler.inverse_transform(predicted_scaled)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))


20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step


In [ ]:
# Filter the data for December 2022 only
december_start = '2022-12-01'
december_end = '2022-12-30'
december_mask = (data.index >= december_start) & (data.index <= december_end)

# Filter the actual and predicted values for December 2022
december_dates = data.index[december_mask]
predicted_december = predicted[-len(december_dates):]
y_test_december = y_test_actual[-len(december_dates):]

# Plotting the actual vs predicted values for December 2022
plt.figure(figsize=(10, 8))
plt.plot(december_dates, y_test_december, label='Actual Close Price', color='red', marker='o')
plt.plot(december_dates, predicted_december, label='LSTM Predicted Close Price', color='green', marker='o')
plt.title('Actual vs LSTM Predicted Close Prices for December 2022')
plt.xlabel('Date')
plt.ylabel('Close Price')

# Format the x-axis to display only date and month
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))
plt.xticks(rotation=45)  # Rotate the labels for better visibility

# Fill between for a clearer comparison
plt.fill_between(december_dates, y_test_december.flatten(), predicted_december.flatten(), 
                 color='gray', alpha=0.2, label='Difference')

plt.legend()
plt.grid(True)
plt.show()


In [10]:
# Calculate and display error metrics for December 2022
mse_dec = mean_squared_error(y_test_december, predicted_december)
mae_dec = mean_absolute_error(y_test_december, predicted_december)

print(f'Mean Squared Error for December 2022: {mse_dec}')
print(f'Mean Absolute Error for December 2022: {mae_dec}')


Mean Squared Error for December 2022: 307.19084607725154
Mean Absolute Error for December 2022: 16.32862176894996
